# 삭제해야 할 패턴들
1. "\xa0"
2. "\n\n "
3. "\n" * m (m > 2)
4. "\s" * m (m > 1)

In [1]:
from transformers import PreTrainedTokenizerFast
import pandas as pd
import re

In [2]:
def data_preprocessing(df):

    # NaN 제거
    dataset = df.dropna(subset=["poem"]) # 현재 poem쪽에 NaN이 존재해서 처리해줌. TODO : 크롤링할 때, poem==NaN이면 skip하도록 구현

    # 중복 제거
    dataset = dataset.drop_duplicates(["poem"], keep="first")

    # 1000자 이상의 데이터 제거
    dataset = dataset[dataset["poem"].str.len() <= 1000]

    # index 재설정
    dataset = dataset.reset_index(drop=True) 

    # 아래의 패턴들을 전처리
    # 1. "\xa0"
    # 2. "\s" * m (m > 1)
    # 3. "\n\n "
    # 4. "\n" * m (m > 2)
    for i in range(len(dataset)):
        dataset.iloc[i, 2] = dataset.iloc[i, 2].replace(u"\xa0", u"")
        dataset.iloc[i, 2] = re.sub(r"( {2,})", "", dataset["poem"][i])
        dataset.iloc[i, 2] = re.sub(r"\n\n ", "", dataset["poem"][i])
        dataset.iloc[i, 2] = re.sub(r"(\n{3,})", "", dataset["poem"][i])

    return dataset

In [4]:
data = pd.read_csv("dataset/crawling_poem.csv")
train_dataset = data_preprocessing(data)

In [39]:
train_dataset["first_line"] = ""
for i in range(len(train_dataset)):
    text_until_newline = re.split('\n', train_dataset["poem"][i], 1)[0]
    train_dataset.iloc[i, 4] = text_until_newline